In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

INPUTS

In [11]:
# a parede é um cubo/quadrado determinado pelo tamanho do lado do cubo/quadrado e numero de dimensoes
tamanho_parede: float      = 10
numero_dimensoes: int      = 3
quantidade_particulas: int = 10

raio:  float = 1
massa: float = 1

velocidade_maxima: float = 10000

PARTICULAS

- Classe das particulas

In [3]:
class Particula:
    def __init__(self, raio: float, massa: float, vetor_posicao: list[float], vetor_velocidade: list[float]) -> None:
        self.massa = massa
        self.raio  = raio
        self.vetor_posicao    = np.array(vetor_posicao)
        self.vetor_velocidade = np.array(vetor_velocidade)

    @property
    def momento(self):
        return self.massa * self.vetor_velocidade
    
    @property
    def energia_cinetica(self):
        return self.massa * self.vetor_velocidade * self.vetor_velocidade
    
    def atualizar_posicao(self, tempo_decorrido: float) -> None:
        self.vetor_posicao += self.vetor_velocidade * tempo_decorrido
    
    def reflexao(self, dimensao_reflexão: int) -> None:
        """
        Edita o vetor velocidade, invertendo o sentido da velocidade na dimensão dada, como uma colisão elástica com a parede.

        Parameters
        ----------
        dimensao_reflexao : int
            0 - Inverte o sentido da velocidade em x\n
            1 - Inverte o sentido da velocidade em y\n
            2 - Inverte o sentido da velocidade em z

        """
        self.vetor_velocidade[dimensao_reflexão] = -self.vetor_velocidade[dimensao_reflexão]

    def __str__(self) -> str:
        return "Raio {}, massa {}, posicao {} e velocidade {}".format(self.raio, self.massa, self.vetor_posicao, self.vetor_velocidade)
    
    def __repr__(self) -> str:
        return "Particula({}, {}, {}, {})".format(self.raio, self.massa, self.vetor_posicao.tolist(), self.vetor_velocidade.tolist())

- RNG
- Criador de vetor aleatorio (com dimensoes certas e sem zeros)

In [4]:
# o RNG é um gerador aleatorio de float entre [0, 1)
rng = np.random.default_rng()

def vetor_aleatorio(valor_minimo: float, valor_maximo: float, dimensoes: int):
    """Gera um vetor (NDArray) com as dimensões dadas sem zeros, para não ter problemas como iniciar já colidindo com a parede"""
    vetor_zero_a_um = rng.random(dimensoes)
    
    while min(vetor_zero_a_um) == 0:
        vetor_zero_a_um[vetor_zero_a_um == 0] = rng.random() # remapeia valores nulos
    
    vetor_aleatorio = (valor_maximo - valor_minimo) * vetor_zero_a_um + valor_minimo

    return vetor_aleatorio

CRIADOR DE LISTA DE PARTICULAS

- Calculador de momento total de uma lista de particulas

In [12]:
def momento_total_lista(lista_de_particulas: list[Particula]):
    momento_total = np.zeros(numero_dimensoes)
    
    for particula in lista_de_particulas:
        momento_total += particula.momento
        
    return momento_total

- Criador de partículas
- Criador da lista de partículas

In [6]:
# criador de particula
def criador_de_particula(raio: float, massa: float) -> Particula:
    posicao_criada = vetor_aleatorio(0, tamanho_parede, numero_dimensoes)
    velocidade_criada = vetor_aleatorio(-velocidade_maxima, velocidade_maxima) # velocidades podem ser positivas ou negativas

    particula_criada = Particula(raio, massa, posicao_criada, velocidade_criada)
    return particula_criada


# criador da lista de partículas
def criar_particulas_iniciais(quantidade_particulas: int, raio: float, massa: float) -> list[Particula]: 
    # primeiro monta uma lista inicial das particulas
    lista_particulas = [criador_de_particula(raio, massa) for i in range(quantidade_particulas)]

    # depois calculamos o momento total das particulas (exceto a ultima)
    momento_total = momento_total_lista(lista_particulas[:-1])
    
    # e fazemos as velocidades da ultima particula equilibrar para o momento total ser zero 
    ultima_particula = lista_particulas[-1]
    velocidade_ultima_particula = - momento_total / ultima_particula.massa
    ultima_particula.vetor_velocidade = velocidade_ultima_particula

    return lista_particulas
        

FUNÇÃO DAS COLISÕES

As colisões podem ser calculadas levando o problema para 1 dimensão (linha da colisão) e retornando para as N dimensões.

TESTAGEM

In [7]:
teste = Particula(1,2,np.array([1,2,3]),np.array([5,10,15]))

In [8]:
teste.reflexao(0)

In [9]:
print(teste.vetor_posicao)
teste.atualizar_posicao(1)
print(teste.vetor_posicao)

[1 2 3]
[-4 12 18]


In [10]:
print(teste.vetor_velocidade)
teste.reflexao(2)
print(teste.vetor_velocidade)

[-5 10 15]
[ -5  10 -15]
